In [ ]:
# One-time setup (only if you didn't run Chapter 2)
%pip -q install ezc3d pandas
!wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
!wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter2Input.py -O Chapter2Input.py
%run Chapter1Input.py
%run Chapter2Input.py




In [ ]:
# Step 0. Helpers that match Chapter 4’s data handling
# Same packages used before
import os, io, base64, json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

P = c3d["parameters"]

def pval(g,k, default=None):
    try:
        return P[g][k]["value"]
    except Exception:
        return default

def get_rates():
    pt_rate = float(pval("POINT","RATE",[np.nan])[0]) if pval("POINT","RATE",None) is not None else np.nan
    an_rate = float(pval("ANALOG","RATE",[np.nan])[0]) if pval("ANALOG","RATE",None) is not None else np.nan
    return pt_rate, an_rate

def reshape_analogs():
    raw = c3d["data"].get("analogs", None)
    if raw is None:
        return np.zeros((0,0))
    A = np.array(raw)
    if A.ndim == 2:
        return A
    if A.ndim == 3:
        # mirror Chapter 4 shape logic
        if A.shape[0] < 16 and A.shape[1] > A.shape[0]:
            return np.moveaxis(A, 0, -1).reshape(A.shape[1], -1)
        return A.reshape(A.shape[0], -1)
    return A.reshape(A.shape[0], -1)

def get_labels():
    point_labels = pval("POINT","LABELS", []) or []
    analog_labels = pval("ANALOG","LABELS", []) or []
    analog_units  = pval("ANALOG","UNITS",  []) or []
    return [str(x) for x in point_labels], [str(x) for x in analog_labels], [str(x) for x in analog_units]



In [ ]:
# Step 1. Find a usable vertical force, just like the Chapter 4 picker
def soft_detrend(y):
    return y - np.nanmedian(y)

def ensure_positive_down(y):
    # choose the sign with larger positive peaks
    pos_peak = np.nanpercentile(np.maximum(y, 0), 99)
    neg_peak = np.nanpercentile(np.maximum(-y, 0), 99)
    return y if pos_peak >= neg_peak else -y

def pick_fz_or_mag(an, labels):
    # prefer any Fz-like label, else build a magnitude from Fx,Fy,Fz
    fz_idx = [i for i,s in enumerate(labels) if "fz" in s.lower() or "vf" in s.lower()]
    for i in fz_idx:
        y = an[i].astype(float)
        y = ensure_positive_down(soft_detrend(y))
        if np.nanmax(np.abs(y)) > 1e-6:
            return y, "Fz", i
    fx = next((i for i,s in enumerate(labels) if "fx" in s.lower()), None)
    fy = next((i for i,s in enumerate(labels) if "fy" in s.lower()), None)
    fz = next((i for i,s in enumerate(labels) if "fz" in s.lower()), None)
    if fx is not None and fy is not None and fz is not None:
        mag = np.sqrt(an[fx]**2 + an[fy]**2 + an[fz]**2).astype(float)
        return soft_detrend(mag), "GRF magnitude", (fx,fy,fz)
    return None, None, None



In [ ]:
# Step 2. Detect stance windows and resample cycles to 0..100 percent
def clean_contact_bool(x, min_len_samples):
    x = x.astype(bool)
    on  = np.where(np.diff(x.astype(int)) == 1)[0] + 1
    off = np.where(np.diff(x.astype(int)) == -1)[0] + 1
    if x[0]:  on  = np.r_[0, on]
    if x[-1]: off = np.r_[off, len(x)]
    y = x.copy()
    for s,e in zip(on,off):
        if e - s < min_len_samples:
            y[s:e] = False
    return y

def detect_stance_pairs(sig, an_rate, thr_frac=0.05, thr_abs_N=20.0, min_stance_s=0.15):
    peak_est = np.nanpercentile(np.abs(sig), 98)
    thr = max(thr_frac * peak_est, thr_abs_N)
    contact = sig > thr
    contact = clean_contact_bool(contact, int(min_stance_s * an_rate))
    starts = np.where(np.diff(contact.astype(int)) == 1)[0] + 1
    ends   = np.where(np.diff(contact.astype(int)) == -1)[0] + 1
    if contact[0] and (len(ends) == len(starts) + 1): starts = np.r_[0, starts]
    if contact[-1] and (len(starts) == len(ends) + 1): ends = np.r_[ends, len(contact)]
    return [(s,e) for s,e in zip(starts,ends) if e > s]

def resample_cycle(y, s, e, n=101):
    if e - s < 3:
        return np.full(n, np.nan)
    xs = np.linspace(s, e - 1, e - s)
    xt = np.linspace(s, e - 1, n)
    return np.interp(xt, xs, y[s:e])



In [ ]:
# Step 3. Compute beginner-friendly metrics that people actually use
# Works whether you have N or 0 cycles. Add body mass if you want BW normalization.
def per_cycle_metrics(sig, t, pairs, mass_kg=None):
    rows = []
    g = 9.81
    for k,(s,e) in enumerate(pairs):
        y = sig[s:e].astype(float)
        tt = t[s:e].astype(float)
        if len(y) < 3:
            continue
        peakN = float(np.nanmax(y))
        pk_idx = int(np.nanargmax(y))
        # crude loading rate: slope from initial rise to 90 percent of peak
        target = 0.9 * peakN
        rise_idx = np.argmax(y > max(1e-9, target))
        rise_idx = max(rise_idx, 1)
        lr = (y[rise_idx] - y[0]) / max(tt[rise_idx] - tt[0], 1e-6)
        contact_time = float(tt[-1] - tt[0])
        out = {
            "Cycle": k+1,
            "PeakN": peakN,
            "ContactTime_s": contact_time,
            "LoadingRate_N_per_s": lr,
        }
        if mass_kg and mass_kg > 0:
            out["Peak_BW"] = peakN / (mass_kg * g)
            out["LoadingRate_BW_per_s"] = lr / (mass_kg * g)
        rows.append(out)
    return pd.DataFrame(rows)

def cadence_from_pairs(pairs, an_rate):
    if len(pairs) < 2:
        return np.nan
    # step events as stance starts
    starts = np.array([s for s,_ in pairs], dtype=float)
    dur_s  = (starts[-1] - starts[0]) / an_rate
    steps  = len(starts)
    return 60.0 * steps / max(dur_s, 1e-6)



In [ ]:
# Step 4. Figures that match Chapter 4 style, plus overlays and bars
def save_fig(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches="tight")
    plt.show()

def fig_time_with_events(t, sig, pairs, title, ylabel, out_path):
    plt.figure(figsize=(8,3))
    plt.plot(t, sig, label="Signal")
    for s,e in pairs:
        plt.axvspan(t[s], t[e-1], alpha=0.2, label="Stance" if s==pairs[0][0] else None)
    plt.xlabel("Time [s]"); plt.ylabel(ylabel); plt.title(title); plt.grid(True, alpha=0.3)
    if pairs: plt.legend()
    save_fig(out_path)

def fig_cycles_overlay(sig, pairs, npts, title, ylabel, out_path):
    if not pairs:
        return None
    X = np.linspace(0,100,npts)
    C = np.array([resample_cycle(sig, s, e, npts) for s,e in pairs])
    meanC = np.nanmean(C, axis=0)
    plt.figure(figsize=(8,3))
    for i in range(C.shape[0]): plt.plot(X, C[i], alpha=0.35)
    plt.plot(X, meanC, linewidth=2, label="Mean")
    plt.xlabel("Gait cycle [%]"); plt.ylabel(ylabel); plt.title(title); plt.grid(True, alpha=0.3); plt.legend()
    save_fig(out_path)
    return C, meanC

def fig_bar_peaks(df, out_path):
    if df is None or df.empty:
        return
    means = df["PeakN"].mean() if "PeakN" in df else np.nan
    plt.figure(figsize=(4,3))
    plt.bar(["All cycles"], [means])
    plt.ylabel("Peak [N]")
    plt.title("Peak GRF average")
    save_fig(out_path)

def fig_scatter_peaks(df, out_path):
    if df is None or df.empty:
        return
    plt.figure(figsize=(5,3))
    plt.scatter(np.arange(len(df))+1, df["PeakN"])
    plt.xlabel("Cycle #"); plt.ylabel("Peak [N]"); plt.title("Per cycle peaks")
    plt.grid(True, alpha=0.3)
    save_fig(out_path)



In [ ]:
# Step 5. Build a tiny report folder with CSV and HTML
def img_to_base64(path):
    with open(path, "rb") as f:
        return "data:image/png;base64," + base64.b64encode(f.read()).decode("ascii")

def write_report(folder, meta, df_metrics, images):
    os.makedirs(folder, exist_ok=True)
    csv_path = os.path.join(folder, "metrics.csv")
    df_metrics.to_csv(csv_path, index=False)

    html_path = os.path.join(folder, "report.html")
    with open(html_path, "w") as f:
        f.write("<h1>Biomechanics Report</h1>")
        f.write("<h3>Summary</h3>")
        f.write("<pre>"+json.dumps(meta, indent=2)+"</pre>")
        f.write("<h3>Metrics</h3>")
        f.write(df_metrics.to_html(index=False))
        for title, p in images:
            if p and os.path.exists(p):
                f.write(f"<h3>{title}</h3>")
                f.write(f'<img src="{img_to_base64(p)}" style="max-width:900px;">')
    return {"csv": csv_path, "html": html_path}



In [ ]:
# Step 6. One clean run
# This block does everything: finds vertical force, detects stance, computes metrics, makes figures, writes the bundle.
# Configs
OUTPUT = "report_outputs"
CYCLE_NPTS = 101
MASS_KG = None  # set to a number if you want BW normalization, for example 75.0

# Data
pt_rate, an_rate = get_rates()
an = reshape_analogs()
point_labels, analog_labels, analog_units = get_labels()
sig, sig_name, src = pick_fz_or_mag(an, analog_labels)

if sig is None or an.shape[1] == 0 or np.isnan(an_rate):
    print("Could not build a reporting signal. Check analog labels and ANALOG.RATE.")
else:
    t_an = np.arange(an.shape[1]) / an_rate
    pairs = detect_stance_pairs(sig, an_rate, thr_frac=0.05, thr_abs_N=20.0, min_stance_s=0.15)
    print(f"Detected {len(pairs)} stance phases using {sig_name}")

    # Metrics
    df = per_cycle_metrics(sig, t_an, pairs, mass_kg=MASS_KG)
    cad = cadence_from_pairs(pairs, an_rate)
    meta = {
        "Signal": sig_name,
        "AnalogRate_Hz": float(an_rate),
        "Cycles": int(len(pairs)),
        "Cadence_spm": None if math.isnan(cad) else float(cad),
        "BW_norm": bool(MASS_KG is not None),
    }

    # Figures
    fig1 = os.path.join(OUTPUT, "time_with_stance.png")
    fig_time_with_events(t_an, sig, pairs, f"{sig_name} over time", f"{sig_name}", fig1)

    cyc = fig_cycles_overlay(sig, pairs, CYCLE_NPTS, f"{sig_name} cycles", f"{sig_name}", os.path.join(OUTPUT, "cycles_overlay.png"))
    fig_bar_peaks(df, os.path.join(OUTPUT, "peaks_bar.png"))
    fig_scatter_peaks(df, os.path.join(OUTPUT, "peaks_scatter.png"))

    # Write bundle
    imgs = [
        ("Time plot with stance", fig1),
        ("Cycle overlay", os.path.join(OUTPUT, "cycles_overlay.png")),
        ("Peak bar", os.path.join(OUTPUT, "peaks_bar.png")),
        ("Peak scatter", os.path.join(OUTPUT, "peaks_scatter.png")),
    ]
    out_paths = write_report(OUTPUT, meta, df, imgs)

    print("Saved:")
    print(" CSV:", out_paths["csv"])
    print(" HTML:", out_paths["html"])
    if df.empty:
        print("Note: no cycles with metrics. Adjust threshold or units if needed.")



In [ ]:
# Optional. Left vs right symmetry, same guardrails as Chapter 4
# If you have separate left and right force channels or separate plates, duplicate the stance detection on each and reuse the cycle overlay.
# Sketch: if you had Fz1 and Fz2
labels = [s.lower() for s in analog_labels]
try:
    fz1 = an[labels.index(next(s for s in labels if "fz1" in s))]
    fz2 = an[labels.index(next(s for s in labels if "fz2" in s))]
    fz1 = ensure_positive_down(soft_detrend(fz1))
    fz2 = ensure_positive_down(soft_detrend(fz2))

    pairs_L = detect_stance_pairs(fz1, an_rate)
    pairs_R = detect_stance_pairs(fz2, an_rate)

    # cycles
    C_L = np.array([resample_cycle(fz1, s, e, CYCLE_NPTS) for s,e in pairs_L]) if pairs_L else np.empty((0, CYCLE_NPTS))
    C_R = np.array([resample_cycle(fz2, s, e, CYCLE_NPTS) for s,e in pairs_R]) if pairs_R else np.empty((0, CYCLE_NPTS))

    if C_L.size and C_R.size:
        X = np.linspace(0,100,CYCLE_NPTS)
        Lm = np.nanmean(C_L, axis=0); Rm = np.nanmean(C_R, axis=0)
        plt.figure(figsize=(8,3))
        plt.plot(X, Lm, label="Left"); plt.plot(X, Rm, label="Right")
        plt.xlabel("Gait cycle [%]"); plt.ylabel("Fz"); plt.title("Symmetry overlay"); plt.grid(True, alpha=0.3); plt.legend()
        save_fig(os.path.join(OUTPUT, "symmetry_overlay.png"))
        # simple symmetry index by area
        aL = np.trapz(Lm, x=X); aR = np.trapz(Rm, x=X)
        if (aL + aR) != 0:
            sym_idx = 100.0 * (aL - aR) / ((aL + aR) / 2.0)
            print(f"Symmetry index [%]: {sym_idx:.2f}")
    else:
        print("Symmetry skipped, missing L or R cycles.")
except StopIteration:
    print("Did not find explicit FZ1 or FZ2 labels, symmetry skipped.")



# Chapter 5. Export / Report

You already loaded data, parsed signals, detected events, and drew the core visuals. Now close the loop. Take the exact signals and cycles you just validated, compute a small set of metrics, save a few clean figures, and drop an HTML plus CSV bundle that anyone can open. The point is not fancy formatting. The point is a repeatable bundle you can reuse across trials and sessions.


## What you will produce

- Per-cycle metrics: peak vertical force, contact time, simple loading rate
- Three figures that match Chapter 4 style: time plot with stance shading, cycle overlay, per-cycle peaks
- An output folder `report_outputs` with `metrics.csv` and `report.html`


---
**Ebook:** *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author:** Dr. Hossein Mokhtarzadeh  
**Powered by:** PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.

> Tip: in Colab or Jupyter, select **Runtime -> Run all**.
---


In [ ]:

# Chapter 5 setup: imports and output folder
import os, math, base64, io, json, textwrap, itertools
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Do not specify styles or colors to keep things simple and portable
OUTPUT_DIR = Path("report_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Outputs will be saved to: {OUTPUT_DIR.resolve()}")


In [ ]:
# Robust auto-detect helpers for time, vertical force, and cycles.
# Works with df columns like "time_s", "Fz (N)", "vGRF", etc.
# You can also pass overrides to get_data_context(df=..., time=..., Fz=..., events=...)

import re
import numpy as np
import pandas as pd

# ---------- small utilities ----------
def _norm(s: str) -> str:
    """Lowercase and strip non-alphanumerics for fuzzy matching."""
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def _choose_name(candidates, pool):
    for c in candidates:
        if c in pool:
            return c
    return None

def _first_existing(names):
    g = globals()
    for n in names:
        if n in g:
            return n, g[n]
    return None, None

# ---------- column picking ----------
_TIME_KEYS = ["time", "times", "time_s", "timesec", "sec", "seconds", "t"]
_FZ_KEYS   = ["fz", "f_z", "verticalforce", "vforce", "grfz", "grf_z", "vgrf", "forcez", "fznewton", "fzn"]

def _pick_cols(df: pd.DataFrame):
    cols = list(df.columns)
    nm = {c: _norm(c) for c in cols}

    # time column by name
    time_col = None
    for c, nc in nm.items():
        if any(k in nc for k in _TIME_KEYS):
            time_col = c
            break

    # fallback time: first numeric column that is mostly increasing
    if time_col is None:
        for c in cols:
            s = pd.to_numeric(df[c], errors="coerce").values
            if np.isfinite(s).sum() > 3:
                d = np.diff(s[np.isfinite(s)])
                if d.size and np.nanmedian(d) > 0:
                    time_col = c
                    break

    # Fz column by name
    fz_col = None
    for c, nc in nm.items():
        if any(k in nc for k in _FZ_KEYS):
            fz_col = c
            break

    # fallback Fz: numeric column with largest range, excluding time
    if fz_col is None:
        num_cols = [c for c in cols if c != time_col and pd.api.types.is_numeric_dtype(df[c])]
        if num_cols:
            rng = []
            for c in num_cols:
                s = pd.to_numeric(df[c], errors="coerce").values
                if np.isfinite(s).sum():
                    rng.append((c, float(np.nanmax(s) - np.nanmin(s))))
            if rng:
                fz_col = max(rng, key=lambda x: x[1])[0]

    return time_col, fz_col

def _find_df_in_globals():
    # preferred names
    df_name, df = _first_existing(["df", "data", "signals", "signal_df"])
    if isinstance(df, pd.DataFrame):
        return df_name, df
    # any DataFrame that looks usable
    for name, obj in globals().items():
        if isinstance(obj, pd.DataFrame):
            tcol, fzcol = _pick_cols(obj)
            if tcol and fzcol:
                return name, obj
    return None, None

# ---------- events to cycles ----------
def _infer_cycles_from_events(n):
    # common arrays
    starts_name, starts = _first_existing([
        "foot_strikes", "heel_strikes", "strikes", "ic_idx", "IC_idx",
        "contact_starts", "hs_idx", "HS_idx"
    ])
    ends_name, ends = _first_existing([
        "toe_offs", "offs", "to_idx", "TO_idx",
        "contact_ends"
    ])

    if starts is not None and ends is not None:
        s = np.asarray(starts).astype(int)
        e = np.asarray(ends).astype(int)
        m = min(len(s), len(e))
        pairs = [(int(s[i]), int(e[i])) for i in range(m)
                 if 0 <= s[i] < n and 0 <= e[i] < n and e[i] > s[i]]
        if pairs:
            pairs.sort(key=lambda p: p[0])
            return pairs, f"{starts_name} + {ends_name}"

    # stance_windows as list of tuples
    sw_name, sw = _first_existing(["stance_windows", "cycles", "contact_windows", "stance_idx"])
    if sw is not None:
        pairs = []
        try:
            for a, b in sw:
                a, b = int(a), int(b)
                if 0 <= a < n and 0 <= b < n and b > a:
                    pairs.append((a, b))
        except Exception:
            pairs = []
        if pairs:
            pairs.sort(key=lambda p: p[0])
            return pairs, f"{sw_name}"

    # dict of events
    ev_name, ev = _first_existing(["events", "evt", "gait_events"])
    if isinstance(ev, dict):
        s = ev.get("start_idx") or ev.get("starts") or ev.get("IC") or ev.get("IC_idx")
        e = ev.get("end_idx")   or ev.get("ends")   or ev.get("TO") or ev.get("TO_idx")
        if s is not None and e is not None:
            s = np.asarray(s).astype(int)
            e = np.asarray(e).astype(int)
            m = min(len(s), len(e))
            pairs = [(int(s[i]), int(e[i])) for i in range(m)
                     if 0 <= s[i] < n and 0 <= e[i] < n and e[i] > s[i]]
            if pairs:
                pairs.sort(key=lambda p: p[0])
                return pairs, f"{ev_name}"

    return None, None

# ---------- threshold fallback ----------
def _fallback_cycles_from_threshold(Fz, thresh_ratio=0.15, min_len=5):
    """Contiguous regions where Fz exceeds thresh_ratio of its max."""
    Fz = np.asarray(Fz)
    if Fz.size == 0 or not np.isfinite(Fz).any():
        return []
    thr = float(np.nanmax(Fz)) * float(thresh_ratio)
    on = Fz > thr
    pairs = []
    i, n = 0, len(on)
    while i < n:
        if on[i]:
            j = i + 1
            while j < n and on[j]:
                j += 1
            if j - i >= min_len:
                pairs.append((i, j - 1))
            i = j
        else:
            i += 1
    return pairs

# ---------- main entry ----------
def _infer_df_and_cols():
    # try explicit names first
    df_name, df = _first_existing(["df", "data", "signals", "signal_df"])
    if isinstance(df, pd.DataFrame):
        tcol, fzcol = _pick_cols(df)
        if tcol and fzcol:
            return df, tcol, fzcol, df_name

    # build from arrays if available
    _, t = _first_existing(["time", "t", "Time_s", "Time"])
    _, fz = _first_existing(["Fz", "FZ", "vertical_force", "GRFz", "grf_z", "fz", "vGRF"])
    if t is not None and fz is not None:
        df = pd.DataFrame({"time": np.asarray(t, dtype=float),
                           "Fz":   np.asarray(fz, dtype=float)})
        return df, "time", "Fz", "(constructed)"

    # scan any DataFrame in globals
    df_name, df = _find_df_in_globals()
    if isinstance(df, pd.DataFrame):
        tcol, fzcol = _pick_cols(df)
        if tcol and fzcol:
            return df, tcol, fzcol, df_name

    return None, None, None, None

def get_data_context(df=None, time=None, Fz=None, events=None):
    """
    Optional overrides:
      df: DataFrame with time and vertical force columns
      time: 1D array of time
      Fz: 1D array of vertical force
      events: dict with keys like start_idx, end_idx, IC, TO
    """
    if df is not None and time is None and Fz is None:
        # pick columns from provided df
        tcol, fzcol = _pick_cols(df)
        if tcol is None or fzcol is None:
            raise RuntimeError("Could not find time and Fz columns in provided df.")
        time_arr = np.asarray(pd.to_numeric(df[tcol], errors="coerce"), dtype=float)
        fz_arr   = np.asarray(pd.to_numeric(df[fzcol], errors="coerce"), dtype=float)
        df_source = "(provided)"
        tcol_name, fz_col_name = tcol, fzcol
    elif time is not None and Fz is not None:
        time_arr = np.asarray(time, dtype=float)
        fz_arr   = np.asarray(Fz, dtype=float)
        df_source = "(from arrays)"
        tcol_name, fz_col_name = "time", "Fz"
    else:
        df_auto, tcol, fzcol, df_source = _infer_df_and_cols()
        if df_auto is None or tcol is None or fzcol is None:
            raise RuntimeError("Could not locate time and vertical force. Define a DataFrame with time and Fz columns or arrays named time and Fz.")
        time_arr = np.asarray(pd.to_numeric(df_auto[tcol], errors="coerce"), dtype=float)
        fz_arr   = np.asarray(pd.to_numeric(df_auto[fzcol], errors="coerce"), dtype=float)
        tcol_name, fz_col_name = tcol, fzcol

    n = len(fz_arr)
    if events is not None and isinstance(events, dict):
        globals()["events"] = events  # make available to _infer_cycles_from_events

    pairs, src = _infer_cycles_from_events(n)
    if not pairs:
        pairs = _fallback_cycles_from_threshold(fz_arr)
        src = f"threshold on {fz_col_name}"

    ctx = {
        "df_source": df_source,
        "time_col": tcol_name,
        "fz_col": fz_col_name,
        "cycle_source": src,
        "time": time_arr,
        "Fz": fz_arr,
        "cycles": pairs
    }
    if len(pairs) == 0:
        print("Warning: no cycles were found. Metrics and plots will be empty until events are available.")
    return ctx

# Example use:
# ctx = get_data_context()                           # auto
# ctx = get_data_context(df=my_df)                   # force a df
# ctx = get_data_context(time=t, Fz=fz)              # force arrays
# ctx = get_data_context(df=my_df, events={"IC": ic_idx, "TO": to_idx})
print("Auto-detect helpers loaded.")


In [ ]:
# Per-cycle metrics with safe context handling.
# Computes:
#   - peak vertical force
#   - contact time
#   - loading rate = (peak - start) / (t_peak - t_start)

import numpy as np
import pandas as pd
from pathlib import Path

def compute_metrics(time, Fz, cycles):
    time = np.asarray(time, dtype=float)
    Fz   = np.asarray(Fz, dtype=float)

    if time.shape[0] != Fz.shape[0]:
        raise ValueError(f"time and Fz lengths differ: {len(time)} vs {len(Fz)}")
    n = len(Fz)

    rows = []
    for k, pair in enumerate(cycles, start=1):
        if pair is None or len(pair) != 2:
            continue
        i0, i1 = int(pair[0]), int(pair[1])

        # keep only valid, forward windows
        if not (0 <= i0 < n and 0 <= i1 < n and i1 > i0):
            continue

        t0 = float(time[i0]); t1 = float(time[i1])
        ct = max(0.0, t1 - t0)

        seg  = Fz[i0:i1+1]
        tseg = time[i0:i1+1]

        # skip empty or all-NaN segments
        if seg.size == 0 or not np.isfinite(seg).any():
            continue

        # peak
        imax = int(np.nanargmax(seg))
        f0   = float(seg[0])
        fpk  = float(seg[imax])
        tpk  = float(tseg[imax])

        # loading rate
        denom = max(1e-9, tpk - t0)
        lr = (fpk - f0) / denom

        rows.append({
            "cycle": k,
            "start_idx": i0,
            "end_idx": i1,
            "start_time_s": t0,
            "end_time_s": t1,
            "contact_time_s": ct,
            "peak_vforce_N": fpk,
            "peak_time_s": tpk,
            "loading_rate_N_per_s": lr
        })

    return pd.DataFrame(rows, columns=[
        "cycle","start_idx","end_idx","start_time_s","end_time_s",
        "contact_time_s","peak_vforce_N","peak_time_s","loading_rate_N_per_s"
    ])

# Get context safely:
try:
    ctx  # noqa: F821
except NameError:
    try:
        # If your earlier cell defined get_data_context(), use it
        ctx = get_data_context()
    except NameError as e:
        raise RuntimeError(
            "No ctx found. Either run the cell that defines get_data_context() "
            "or call compute_metrics(time, Fz, cycles) with your own arrays."
        ) from e

# Compute and save
metrics_df = compute_metrics(ctx["time"], ctx["Fz"], ctx["cycles"])
print(f"Computed {len(metrics_df)} cycles")

OUTPUT_DIR = Path("report_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

csv_path = OUTPUT_DIR / "metrics.csv"
metrics_df.to_csv(csv_path, index=False)
print("Saved:", csv_path.resolve())


In [ ]:

# Create the three figures:
# 1) Time plot with stance shading
# 2) Cycle overlay
# 3) Per-cycle peaks

time = ctx["time"]
Fz   = ctx["Fz"]
cycles = ctx["cycles"]

# 1) Time plot with stance shading
plt.figure()
plt.plot(time, Fz, linewidth=1.5)
for (i0, i1) in cycles:
    t0, t1 = time[i0], time[i1]
    plt.fill_between([t0, t1], [max(Fz.min(), 0)], [max(Fz)], alpha=0.15)
plt.xlabel("Time [s]")
plt.ylabel("Vertical Force [N]")
plt.title("Vertical Force over Time with Stance Shading")
fig1_path = OUTPUT_DIR / "fig_time.png"
plt.savefig(fig1_path, dpi=150, bbox_inches="tight")
plt.close()
print("Saved:", fig1_path.resolve())

# 2) Cycle overlay: align each cycle to its own t=0 and plot Fz
plt.figure()
for (i0, i1) in cycles:
    tseg = time[i0:i1+1]
    fseg = Fz[i0:i1+1]
    if len(tseg) < 2:
        continue
    tnorm = tseg - tseg[0]
    plt.plot(tnorm, fseg, linewidth=1.0, alpha=0.9)
plt.xlabel("Time from contact start [s]")
plt.ylabel("Vertical Force [N]")
plt.title("Cycle Overlay - Vertical Force")
fig2_path = OUTPUT_DIR / "fig_overlay.png"
plt.savefig(fig2_path, dpi=150, bbox_inches="tight")
plt.close()
print("Saved:", fig2_path.resolve())

# 3) Per-cycle peaks: bar chart of peak vertical force
plt.figure()
if len(cycles) > 0 and len(metrics_df):
    peaks = metrics_df["peak_vforce_N"].values
    x = np.arange(1, len(peaks) + 1)
    plt.bar(x, peaks)
    plt.xlabel("Cycle")
    plt.ylabel("Peak Vertical Force [N]")
    plt.title("Per-cycle Peak Vertical Force")
else:
    plt.text(0.5, 0.5, "No cycles available", ha="center", va="center")
fig3_path = OUTPUT_DIR / "fig_peaks.png"
plt.savefig(fig3_path, dpi=150, bbox_inches="tight")
plt.close()
print("Saved:", fig3_path.resolve())


In [ ]:

# Write simple report.html that links to metrics.csv and embeds the figures using string.Template
from datetime import datetime
from string import Template

fig1_rel = "fig_time.png"
fig2_rel = "fig_overlay.png"
fig3_rel = "fig_peaks.png"

tpl = Template(
    "<!DOCTYPE html>\n"
    "<html lang=\"en\">\n"
    "<head>\n"
    "  <meta charset=\"utf-8\">\n"
    "  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n"
    "  <title>Chapter 5 Report - PoseIQ</title>\n"
    "  <style>\n"
    "    body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; line-height: 1.5; margin: 24px; }\n"
    "    header { margin-bottom: 16px; }\n"
    "    h1, h2, h3 { margin: 0.4em 0; }\n"
    "    .meta { color: #444; font-size: 0.95rem; }\n"
    "    .card { border: 1px solid #ddd; border-radius: 12px; padding: 16px; margin: 16px 0; }\n"
    "    img { max-width: 100%; height: auto; display: block; margin: 12px 0; }\n"
    "    a.button { display: inline-block; padding: 8px 12px; border: 1px solid #333; border-radius: 8px; text-decoration: none; color: #111; }\n"
    "  </style>\n"
    "</head>\n"
    "<body>\n"
    "  <header>\n"
    "    <h1>Export and Report</h1>\n"
    "    <div class=\"meta\">Generated $stamp</div>\n"
    "    <div class=\"meta\">Source columns: time = $time_col, vertical force = $fz_col from $df_source</div>\n"
    "    <div class=\"meta\">Cycles: $cycle_count via $cycle_source</div>\n"
    "  </header>\n"
    "\n"
    "  <section class=\"card\">\n"
    "    <h2>Metrics</h2>\n"
    "    <p>Download the CSV with per-cycle metrics.</p>\n"
    "    <p><a class=\"button\" href=\"metrics.csv\" download>metrics.csv</a></p>\n"
    "  </section>\n"
    "\n"
    "  <section class=\"card\">\n"
    "    <h2>Figures</h2>\n"
    "    <h3>Time plot with stance shading</h3>\n"
    "    <img src=\"$fig1\" alt=\"Vertical Force over Time with Stance Shading\">\n"
    "    <h3>Cycle overlay</h3>\n"
    "    <img src=\"$fig2\" alt=\"Cycle Overlay - Vertical Force\">\n"
    "    <h3>Per-cycle peak vertical force</h3>\n"
    "    <img src=\"$fig3\" alt=\"Per-cycle peak vertical force\">\n"
    "  </section>\n"
    "\n"
    "  <footer class=\"card\">\n"
    "    <div><strong>Ebook:</strong> A Hands-On Guide to Biomechanics Data Analysis with Python and AI</div>\n"
    "    <div><strong>Author:</strong> Dr. Hossein Mokhtarzadeh</div>\n"
    "    <div><strong>Powered by:</strong> PoseIQ™</div>\n"
    "  </footer>\n"
    "</body>\n"
    "</html>\n"
)

html = tpl.safe_substitute(
    stamp=datetime.now().isoformat(timespec="seconds"),
    time_col=ctx["time_col"],
    fz_col=ctx["fz_col"],
    df_source=ctx["df_source"],
    cycle_count=len(ctx["cycles"]),
    cycle_source=ctx["cycle_source"],
    fig1=fig1_rel,
    fig2=fig2_rel,
    fig3=fig3_rel,
)

html_path = OUTPUT_DIR / "report.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html)

print("Saved:", html_path.resolve())


You are done. The `report_outputs` folder now holds `metrics.csv` and `report.html` along with three figures. If cycles are empty, confirm earlier chapters produced stance events or adjust the threshold in the fallback code cell.
